In [ ]:
%sql
create database if not exists finance_data_catalog

%sql
use finance_data_catalog

# Creating Transaction Table
%sql
create table if not exists transaction_data(TransactionID String,CustomerID String, TransactionAmount INT, TransactionDate Date)

#creating Customer_table
%sql
create table if not exists customer_data(CustomerID String,CustomerName String, Email String, Country String)

In [ ]:
# meta data retrieval
%sql
show tables in finance_data_catalog;

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinanceDataCatalog").getOrCreate()
#profiling

# Profile transaction amounts
spark.sql("""
    SELECT
        MIN(TransactionAmount) AS MinAmount,
        MAX(TransactionAmount) AS MaxAmount,
        AVG(TransactionAmount) AS AvgAmount,
        COUNT(*) AS TotalTransactions
    FROM schema_name.transaction_data
""").show()

# Profile customer locations
spark.sql("""
    SELECT
        CustomerLocation,
        COUNT(*) AS NumTransactions
    FROM schema_name.transaction_data
    GROUP BY CustomerLocation
""").show()


In [ ]:
# merging the data
merged_df = spark.sql("""
    SELECT
        t.TransactionID,
        t.TransactionAmount,
        t.TransactionDate,
        c.CustomerID,
        c.CustomerName,
        c.Email,
        c.CustomerLocation
    FROM schema_name.transaction_data t
    JOIN schema_name.customer_data c
    ON t.CustomerID = c.CustomerID
""")
merged_df.show()

In [ ]:
%sql
grant all PRIVILEGES on table finance_data_catalog.Transaction_data to 'DataEngineers';

%sql
grant all PRIVILEGES on table finance_data_catalog.Customer_data to 'DataEngineers';

%sql
grant select on table finance_data_catalog.Customer_data to 'DataAnalysts';

%sql
grant select(TransactionID,TransactionDate,TransactiionAmount) on table finance_data_catalog.Customer_data to 'DataEngineers';

In [ ]:

spark.sql('''SELECT COUNT(*)
FROM schema_name.transaction_data
WHERE TransactionAmount < 0;
''').show()

spark.sql('''
SELECT COUNT(*)
FROM schema_name.customer_data
WHERE Email NOT LIKE '%_@__%.__%';
''').show()

In [ ]:
spark.sql("vacuum finance_data_catalog.transaction_data retain 168 hours")
spark.sql("vacuum finance_data_catalog.customer_data retain 168 hours")

**Task -2**

In [ ]:
%sql
create database if not exists corporate_data_catalog

%sql
use corporate_data_catalog

%sql
create table if not exists sales_data(SalesID String, CustomerID String,SalesAmount INT, SalesDate DATE);

%sql
create table if not exists hr_data (EmployeeID STRING, EmployeeName String,Department String, Salary INT);

%sql
create table if not exists finance_data(InvoiceID String,VendorID String,InvoiceAmount INT,PaymentDate INT);


In [ ]:
#tagging sensitive data
%sql
CREATE TAG SensitiveData;

%sql
ALTER TABLE hr_data.employee ADD TAG SensitiveData ON COLUMN Salary;

%sql
ALTER TABLE finance_data.invoice ADD TAG SensitiveData ON COLUMN InvoiceAmount;

spark.sql(''')


In [ ]:
# provisioning
spark.sql('''
select max(SalesAmount) as max_sales,sum(SalesAmount) as total_sales from schema_name.sales_data
'''
)
spark.sql('''
select max(Salary) as max_salary,sum(Salary) as total_salary from schema_name.hr_data
''')
spark.sql('''
select max(InvoiceAmount) as max_invoice,sum(InvoiceAmount) as total_invoice from schema_name.finance_data
''')

In [ ]:
# privileges to different persons according to their roles
%sql
grant all PRIVILEGES on table corporate_data_catalog.sales_data to 'SalesTeam';

%sql
revoke all PRIVILEGES on table corporate_data_catalog.hr_data from 'SalesTeam';

%sql
revoke all PRIVILEGES on table corporate_data_catalog.finance_data from 'SalesTeam;

%sql
grant select PRIVILEGE on table corporate_data_catalog.hr_data to 'HRTeam';

%sql
grant update privilege on table corporate_data_catalog.hr_data to 'HRTeam';

%sql
grant all privileges on table corporate_data_catalog.finance_data to 'FinanceTeam';

#column level security

%sql
grant select(SalesID,SalesAmount,SalesDate) on table corporate_data_catalog.sales_data to 'SalesTeam';

%sql
grant select(EmployeeID,EmployeeName,Department) on table corporate_data_catalog.hr_data to 'HRTeam

%sql
grant select(InvoiceID,InvoiceAmount,PaymentDate) on table corporate_data_catalog.finance_data to 'FinanceTeam';


In [ ]:
#data quality assurance
%sql
ALTER TABLE sales_data.sales ADD CONSTRAINT check_sales_amount_positive CHECK (SalesAmount > 0);

%sql
ALTER TABLE hr_data.employee ADD CONSTRAINT check_salary_positive CHECK (Salary > 0);

%sql
ALTER TABLE finance_data.invoice ADD CONSTRAINT check_invoice_amount_positive CHECK (InvoiceAmount > 0);


In [ ]:
spark.sql("optimize corporate_data_catalog.sales_data")
spark.sql("optimize corporate_data_catalog.finance_data")
spark.sql("optimize corporate_data_catalog.hr_data")
spark.sql("vacuum corporate_data_catalog.sales_data retain 168 hours")
spark.sql("vacuum corporate_data_catalog.finance_data retain 168 hours")
spark.sql("vacuum corporate_data_catalog.hr_data retain 168 hours")

**Task-3**`

In [ ]:
%sql
create database if not exists enterprise_data_catalog

%sql
create table if not exists marketing_data(CampaignID String,CampaignName String, Budget INT, StartDate Date);

%sql
create table if not exists  operations_data( OrderID String,ProductID String, Quantity INT, ShippingStatus String);

%sql
create table if not exists it_data(IncidentID String,ReportedBy String, IssueType String, ResolutionTime DateTime);


In [ ]:
%sql
%sql
CREATE TAG SensitiveData1;

%sql
ALTER TABLE enterprise_data_catalog.marketing_data ADD TAG SensitiveData1 ON COLUMN Budget;

%sql
ALTER TABLE enterprise_data_catalog.marketing_data ADD TAG SensitiveData1 ON COLUMN Budget;

%sql
ALTER TABLE enterprise_data_catalog.operations_data ADD TAG SensitiveData1 ON COLUMN Quantity;

%sql
ALTER TABLE enterprise_data_catalog.it_data ADD TAG SensitiveData1 ON COLUMN ResolutionTime;


In [ ]:
#provisioningb
from pyspark.sql.functions import year, month

# Add year and month columns
df_marketing = df_marketing.withColumn("Year", year(df_marketing["StartDate"]))
df_marketing = df_marketing.withColumn("Month", month(df_marketing["StartDate"]))

# Group by year and month for budget trends
df_marketing.groupBy("Year", "Month").agg({"Budget": "sum"}).orderBy("Year", "Month").show()

# Load operational data
df_operational = spark.table("operational_data")

# Count occurrences of each shipping status
df_operational.groupBy("ShippingStatus").count().orderBy("count", ascending=False).show()
# Add year and month columns
df_operational = df_operational.withColumn("Year", year(df_operational["ShippingDate"]))
df_operational = df_operational.withColumn("Month", month(df_operational["ShippingDate"]))

# Group by year and month to see shipping trends
df_operational.groupBy("Year", "Month").count().orderBy("Year", "Month").show()
# Filter and count delayed shipments
delayed_shipments = df_operational.filter(df_operational["ShippingStatus"] == "Delayed")
delayed_count = delayed_shipments.count()




In [ ]:
%sql
grant all privileges on table enterprise_data_catalog.marketing_data to 'MarketingTeam';

%sql
revoke all privileges on table enterprise_data_catalog.it_data from 'MarketingTeam';

%sql
revoke all privileges on table enterprise_data_catalog.operations_data from 'MarketingTeam';

%sql
grant all privileges on table enterprise_data_catalog.operations_data to 'OperationsTeam';

%sql
grant select privileges on table enterprise_data_catalog.matketing_data to 'OperationsTeam';

%sql
grant all privileges on table enterprise_data_catalog.it_data to 'ITTeam';

%sql
grant update privileges on table enterprise_data_catalog.it_data to 'ITTeam';

#column level security
 grant select(CampaignID,CampaignName,StartDate) on table enterprise_data_catalog.marketing_data to 'MarketingTeam';

%sql
grant select(OrderID,ProductID,ShippingStatus) on table enterprise_data_catalog.operations_data to 'OperationsTeam';

%sql
grant select(IncidentID,ReportedBy,IssueType) on table enterprise_data_catalog.it_data to 'ITTeam';


In [ ]:
%sql
ALTER TABLE marketing_data ADD CONSTRAINT check_budget_positive CHECK (Budget > 0);

df_marketing = spark.table("marketing_data")
# Validate that all campaign budgets are greater than zero
invalid_budget = df_marketing.filter(df_marketing["Budget"] <= 0)
invalid_budget.show()

%sql
ALTER TABLE operations_data ADD CONSTRAINT check_shipping_status CHECK (ShippingStatus IN ('Pending', 'Shipped', 'Delivered'));
df_operations = spark.table("operations_data")

%sql
ALTER TABLE it_data ADD CONSTRAINT check_resolution_time_non_negative CHECK (ResolutionTime >= 0);

df_it = spark.table("it_data")
invalid_resolution_time = df_it.filter(df_it["ResolutionTime"] < 0)


In [ ]:
spark.sql("optimize enterprise_data_catalog.marketing_data")
spark.sql("optimize enterprise_data_catalog.operations_data")
spark.sql("optimize enterprise_data_catalog.it_data")

spark.sql(''' vacuum enterprise_data_catalog.marketing_data retain 168 hours''')
spark.sql(''' vacuum enterprise_data_catalog.operations_data retain 168 hours''')
spark.sql(''' vacuum enterprise_data_catalog.it_data retain 168 hours''')


**Task-4**

In [ ]:
import pandas as pd

try:
  df_weather=pd.read_csv("weather.csv")
  print(df)
except Exception as e:
  print("The file doesnt exists")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("WeatherData").getOrCreate()

dbutils.fs.cp("file:workspace/shared/weather.csv","dbfs:filestore/weather.csv")
df_weather=spark.read.format("csv").load("dbfs:filestore/weather.csv").option("header","True").option("inferSchema","True")
df_weather.write.format("delta").save("dbf;/delta/weather_delta")
#Cleaning Data
df_weather=spark.read.format("delta").load("dbfs:/delta/weather_delta")
df_weather1=df_weather.drop.isna()
df_weather1.write.format("delta").save("dbfs:/delta/weather_delta_new")


In [ ]:
import DLT

@dlt.table
def transformed_weather_data():
    df_cleaned = dlt.read("df_weather1")
    # Perform transformations
    try:
        df_transformed = (
            df_cleaned
            .withColumn("avg_Humidity", F.avg(F.col("Humidity")).over())
            .withColumn("avg_Temperature", F.avg(F.col("Temperature")).over())
        )
        dlt.log("Data transformation successful")
        return df_transformed
    except Exception as e:
        dlt.log(f"Data transformation failed: {str(e)}")
        raise Exception(f"Failed to transform weather data: {e}")

In [ ]:
import pandas as pd

try:
  df_transactions=pd.read_csv("transactions.csv")
  print(df_transactions)
except Exception as e:
  print("The file doesnt exists")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("transactions").getOrCreate()

df_transactions=spark.read.format("csv").load("dbfs:filestore/transactions.csv").option("header","True").option("inferSchema","True")
df_transactions.write.format("delta").save("dbfs:/delta/transactions_delta")

dbutils.fs.cp("file:workspace/shared/transactions.csv","dbfs:filestore/transactions.csv")
# Task 2: Data Cleaning
@dlt.table
def clean_weather_data():
    """
    Clean weather data by removing null values.
    """
    df_weather = dlt.read("df_transactions")
    try:
        df_cleaned = df_weather.dropna()
        dlt.log("Data cleaning successful.")
        return df_cleaned
    except Exception as e:
        dlt.log(f"Data cleaning failed: {str(e)}")
        raise Exception(f"Failed to clean weather data: {e}")


# Task 3: Data Transformation
@dlt.table
def transform_weather_data():
    """
    Transform the cleaned weather data by adding average humidity and temperature columns.
    """
    df_cleaned = dlt.read("clean_weather_data")
    try:
        # Add new columns with the average of Humidity and Temperature
        df_transformed = (
            df_cleaned
            .withColumn("avg_Humidity", F.avg(F.col("Humidity")).over())
            .withColumn("avg_Temperature", F.avg(F.col("Temperature")).over())
        )
        dlt.log("Data transformation successful.")
        return df_transformed
    except Exception as e:
        dlt.log(f"Data transformation failed: {str(e)}")
        raise Exception(f"Failed to transform weather data: {e}")


In [ ]:
dbutils.fs.cp("file:workspace/shared/inventory.csv","dbfs:/filestore/inventory.csv")
try:
  df_inventory=pd.read_csv("inventory.csv")
  print(df_inventory)
except Exception as e:
  print("The file doesnt exists")
df_inventory=spark.read.format("csv").load("dbfs:/filestore/inventory.csv").option("header","True").option("inferSchema","True")
df_inventory.write.format("delta").save("dbfs:/delta/inventory_delta")

@dlt.table
def clean_inventory_data():
    """
    Clean inventory data by handling missing values or duplicates.
    """
    df_inventory = dlt.read("ingest_inventory_data")
    try:
        # Drop rows with null values
        df_cleaned = df_inventory.dropna()
        # Optionally, you can remove duplicates
        df_cleaned = df_cleaned.dropDuplicates()
        dlt.log("Inventory data cleaning successful.")
        return df_cleaned
    except Exception as e:
        dlt.log(f"Inventory data cleaning failed: {str(e)}")
        raise Exception(f"Failed to clean inventory data: {e}")

@dlt.table
def transform_inventory_data():
    """
    Transform the cleaned inventory data by performing aggregations or calculating new columns.
    """
    df_cleaned = dlt.read("clean_inventory_data")
    try:
        # Example transformation: Adding a new column for total value based on quantity and price
        df_transformed = df_cleaned.withColumn("total_value", F.col("quantity") * F.col("price"))

        dlt.log("Inventory data transformation successful.")
        return df_transformed
    except Exception as e:
        dlt.log(f"Inventory data transformation failed: {str(e)}")
        raise Exception(f"Failed to transform inventory data: {e}")


In [ ]:
dbutils.fs.cp("file:workspace/shared/employ.csv","dbfs:/filestore/employ.csv")
try:
  df_employ=pd.read_csv("employ.csv")
  print(df_employ)
except Exception as e:
  print("The file doesnt exists")
df_employ=spark.read.format("csv").load("dbfs:/filestore/employ.csv").option("header","True").option("inferSchema","True")
df_employ.write.format("delta").save("dbfs:/delta/inventory_delta")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, dayofmonth, month, year

spark = SparkSession.builder.appName("AttendanceSummary").getOrCreate()
attendance_df = spark.read.format("delta").load("dbfs:/delta/attendance_data")

current_month = 9
current_year = 2024

filtered_df = attendance_df.filter(month(col("date")) == current_month) & (year(col("date")) == current_year))
total_hours_df = filtered_df.groupBy("employee_id").agg(spark_sum("hours_worked").alias("total_hours"))
overtime_df = filtered_df.filter(col("hours_worked") > 8)
total_hours_df.write.format("delta").mode("overwrite").save("dbfs:/delta/attendance_summary")
overtime_df.write.format("delta").mode("overwrite").save("dbfs:/delta/overtime_summary")

import dlt
from pyspark.sql.functions import col, sum as spark_sum, dayofmonth, month, year
# Step 1: Data Ingestion
@dlt.table
def ingest_attendance_data():
    return spark.read.format("delta").load("dbfs:/delta/attendance_data")
# Step 2: Data Cleaning
@dlt.table
def clean_attendance_data():
    attendance_df = dlt.read("ingest_attendance_data")
    # Example cleaning: Drop rows with missing values
    return attendance_df.dropna()
# Step 3: Attendance Summary
@dlt.table
def attendance_summary():
    attendance_df = dlt.read("clean_attendance_data")

    current_month = 9  # September
    current_year = 2024

    filtered_df = attendance_df.filter(
        (month(col("date")) == current_month) & (year(col("date")) == current_year)
    )

    # Calculate total hours worked by each employee
    return filtered_df.groupBy("employee_id").agg(spark_sum("hours_worked").alias("total_hours"))

# Step 4: Overtime Detection
@dlt.table
def overtime_summary():
    attendance_df = dlt.read("clean_attendance_data")
    return attendance_df.filter(col("hours_worked") > 8)

# View the history of changes made to the Delta table
spark.sql("DESCRIBE HISTORY delta.`dbfs:/delta/attendance_data`").show(truncate=False)

